In [11]:
import numpy as np
import matplotlib.pyplot as plt
from pyomo.environ import *
from pyomo.dae import *
import pandas as pd
from get_sensitivity import *
#from pyomo.contrib.sensitivity_toolbox.sens import sensitivity_calculation, get_dsdp, get_dfds_dcds
from idaes.apps.uncertainty_propagation.sens import get_dsdp, get_dfds_dcds

In [2]:
def reactor_run(dv=[1.0, 300], param_init=[84.79085853498033, 371.71773413976416, 
                                7.777032028026428, 15.047135137500822]):
    '''
    This function contains the reactor kinetics model.
    
    Arguments:
        dv: design variable values, [CA_init: initial value of CA [M], T_init: initial temperature [K]]
        para_samp: parameter values, size [# of sample]
        
    Return: 
        m: reactor kinetics model
    '''
    # time points 
    timepoint = [0.0, 0.125, 0.25, 0.375, 0.5,0.625, 0.75, 0.875, 1.0]
 
    # parameter names
    para_list = ['A1', 'A2', 'E1', 'E2']
    
    # names of observed variables
    y_list = ['CA','CB','CC']
    
    ### Create mathematical model
    m = ConcreteModel()
    
    ## Define experimental design variables
    # Initial concentration of CA0S
    m.CA0 = Var(initialize = dv[0], bounds=(1.0,5.0), within=NonNegativeReals) # mol/L
    m.CA0.fix()
    
    # Reactor temperature (constant)
    m.T = Var(initialize = dv[1], bounds=(300, 700), within=NonNegativeReals)  
    m.T.fix()
    
    # Define ideal gas constant
    m.R = 8.31446261815324 # J / K / mole
    

    ### Define sets and expressions
    # timepoint
    m.t = Set(initialize=timepoint)
    
    # parameter index
    m.para = Set(initialize=para_list)
    
    # response/observed variables
    m.y = Set(initialize=y_list)
    
    # Concentration of A, B, and C [mol/L]
    m.C = Var(m.y, m.t, initialize=0.0, within=NonNegativeReals)
    
    # Define parameters
    m.A1 = Param(initialize=param_init[0],mutable=True)
    m.A2 = Param(initialize=param_init[1],mutable=True)
    m.E1 = Param(initialize=param_init[2],mutable=True)
    m.E2 = Param(initialize=param_init[3],mutable=True)
    
    
    m.A1_pert = Param(initialize=param_init[0]*1.1)
    m.A2_pert = Param(initialize=param_init[1])
    m.E1_pert = Param(initialize=param_init[2])
    m.E2_pert = Param(initialize=param_init[3])
    
    
    ### Define the mathematical model
    # Rate law for reaction 1 (Arrhenius equation)
    def k1_rule(m):
        return m.A1 * exp(-m.E1*1000/(m.R*m.T))
    
    m.k1 = Expression(rule = k1_rule) # 1/hr
    
    # Rate law for reaction 2
    def k2_rule(m):
        return m.A2 * exp(-m.E2*1000/(m.R*m.T))
    
    m.k2 = Expression(rule = k2_rule) # 1/hr
    
    # Calculate model response variables
    def conc(m,i,t):
        '''
        Calculate the model predictions
        Argument: 
            i: the model responses, CA, CB, CC
            t: timepoints
        '''
        if i == 'CA':
            return m.C['CA',t] == m.CA0*exp(-m.k1*t)
        elif i == 'CB':
            return m.C['CB',t] == m.k1*m.CA0/(m.k2-m.k1) * (exp(-m.k1*t) - exp(-m.k2*t))
        else:
            return m.C['CC',t] == m.CA0 - m.C['CA',t] - m.C['CB',t]
 
    m.rate = Constraint(m.y, m.t, rule=conc)
    
    def obj_rule(m):
        return 0
    
    m.Obj = Objective(rule=obj_rule, sense= minimize)
    
    m.A1.setlb(param_init[0])
    m.A1.setub(param_init[0])
    
    m.A2.setlb(param_init[1])
    m.A2.setub(param_init[1])
    
    m.E1.setlb(param_init[2])
    m.E1.setub(param_init[2])
    
    m.E2.setlb(param_init[3])
    m.E2.setub(param_init[3])
    
    
    # Return Pyomo model
    return m

## Test k_aug

In [ ]:
param_initial = [84.79085853498033, 371.71773413976416, 7.777032028026428, 15.047135137500822]


mod = reactor_run(param_init=param_initial)

solver = SolverFactory('ipopt')
solver.solve(mod, tee=True)

In [ ]:
print(value(eval('mod.C["CA", 0.125]')))

In [ ]:



list_original = [mod.A1,mod.A2,mod.E1,mod.E2]
list_perturb = [mod.A1_pert, mod.A2_pert, mod.E1_pert, mod.E2_pert]


#m_sipopt = sensitivity_calculation('sipopt', mod, list_original, list_perturb, tee=True)
m_kaug_dsdp = sensitivity_calculation('k_aug', mod, list_original, list_perturb, tee=True)

In [ ]:
#x = eval('m_sipopt.sens_sol_state_1[m_sipopt.C["CB",0.125]]')
y = eval('m_kaug_dsdp.C["CB",0.125]()')

print(y)

In [ ]:
print(m_kaug_dsdp.C["CB",0.125]())
print(m_kaug_dsdp.C['CB',0]())

In [5]:
# Parameter name
variable_name = ['A1', 'A2','E1','E2']
# Parameter initial value
theta_v = [84.79085853498033, 371.71773413976416, 7.777032028026428, 15.047135137500822]

In [6]:
def create_model(CA_init=1, T_init=300):
    '''Model for get_sensitivity, defining measurements as cosntraints
    '''
    
    # time points 
    timepoint = [0.0, 0.125, 0.25, 0.375, 0.5,0.625, 0.75, 0.875, 1.0]
    
    # names of observed variables
    y_list = ['CA','CB','CC']
    
    m= ConcreteModel()

    m.CA0 = CA_init
    m.T = T_init
    
    # Define ideal gas constant
    m.R = 8.31446261815324 # J / K / mol

    ### Define sets and expressions
    # timepoint
    m.t = Set(initialize=timepoint)
    
    # response/observed variables
    m.y = Set(initialize=y_list)

    m.A1 = Var(initialize = theta_v[0])
    m.A2 = Var(initialize = theta_v[1])
    m.E1 = Var(initialize = theta_v[2])
    m.E2 = Var(initialize = theta_v[3])
    
    # Concentration of A, B, and C [mol/L]
    m.C = Var(m.y, m.t, initialize=0.0, within=NonNegativeReals)
    m.dCdt = Var(m.y, m.t, initialize=0.0, within=NonNegativeReals)

    def k1_rule(m):
        return m.A1*exp(-m.E1*1000/m.R/m.T)

    def k2_rule(m):
        return m.A2 * exp(-m.E2*1000/m.R/m.T)

    m.k1 = Expression(rule = k1_rule) # 1/hr
    m.k2 = Expression(rule = k2_rule)
    
    def conc(m,i,t):
        '''
        Calculate the model predictions
        Argument: 
            i: the model responses, CA, CB, CC
            t: timepoints
        '''
        if i == 'CA':
            return m.C['CA',t] == m.CA0*exp(-m.k1*t)
        elif i == 'CB':
            return m.C['CB',t] == m.k1*m.CA0/(m.k2-m.k1) * (exp(-m.k1*t) - exp(-m.k2*t))
        else:
            return m.C['CC',t] == m.CA0 - m.CA0*exp(-m.k1*t) - m.k1*m.CA0/(m.k2-m.k1) * (exp(-m.k1*t) - exp(-m.k2*t))

    m.rate = Constraint(m.y, m.t, rule=conc)
    
    def gradient(m, i, t):
        if i=='CA':
            return m.dCdt['CA',t] == -m.k1*m.C['CA',t]
        elif i=='CB':
            return m.dCdt['CB',t] == m.k1*m.C['CA',t] - m.k2*m.C['CB',t]
        elif i=='CC':
            return m.dCdt['CC',t] == m.k2*m.C['CB',t]
        
    #m.grad = Constraint(m.y, m.t, rule=gradient)

    def obj_rule(m):
        return 0
    
    m.Obj = Objective(rule=obj_rule, sense=maximize)
    
    theta_dict = {'A1': theta_v[0], 'A2': theta_v[1], 'E1': theta_v[2], 'E2':theta_v[3]}
    for v in variable_name:
        getattr(m, v).setlb(theta_dict[v])
        getattr(m, v).setub(theta_dict[v])
        
    return m

In [12]:
# create the model
model_un = create_model(CA_init=2, T_init=400)
# get information
gradient_f, gradient_c, line_dic =  get_sensitivity(model_un, variable_name)
#gradient_f, gradient_c, col,row, line_dic =  get_dfds_dcds(model_un, variable_name, tee=True)

Ipopt 3.13.2: linear_solver=ma57


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use of the HSL codes within IPOPT must
    contain the following acknowledgement:
        HSL, a collection of Fortran codes for large-scale scientific
        compu

In [13]:
print(col)
print(row)
print(line_dic)

['A1', 'A2', 'E1', 'E2', 'C[CA,0.0]', 'C[CA,0.125]', 'C[CA,0.25]', 'C[CA,0.375]', 'C[CA,0.5]', 'C[CA,0.625]', 'C[CA,0.75]', 'C[CA,0.875]', 'C[CA,1.0]', 'C[CB,0.0]', 'C[CB,0.125]', 'C[CB,0.25]', 'C[CB,0.375]', 'C[CB,0.5]', 'C[CB,0.625]', 'C[CB,0.75]', 'C[CB,0.875]', 'C[CB,1.0]', 'C[CC,0.0]', 'C[CC,0.125]', 'C[CC,0.25]', 'C[CC,0.375]', 'C[CC,0.5]', 'C[CC,0.625]', 'C[CC,0.75]', 'C[CC,0.875]', 'C[CC,1.0]']
['rate[CA,0.125]', 'rate[CA,0.25]', 'rate[CA,0.375]', 'rate[CA,0.5]', 'rate[CA,0.625]', 'rate[CA,0.75]', 'rate[CA,0.875]', 'rate[CA,1.0]', 'rate[CB,0.125]', 'rate[CB,0.25]', 'rate[CB,0.375]', 'rate[CB,0.5]', 'rate[CB,0.625]', 'rate[CB,0.75]', 'rate[CB,0.875]', 'rate[CB,1.0]', 'rate[CC,0.125]', 'rate[CC,0.25]', 'rate[CC,0.375]', 'rate[CC,0.5]', 'rate[CC,0.625]', 'rate[CC,0.75]', 'rate[CC,0.875]', 'rate[CC,1.0]', 'rate[CA,0.0]', 'rate[CB,0.0]', 'rate[CC,0.0]', 'Obj']
{'A1': 1, 'A2': 2, 'E1': 3, 'E2': 4}


In [14]:
print(gradient_c)



[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 8.67515581e-03  0.00000000e+00 -2.21173016e-01  0.00000000e+00]
 [ 6.24023430e-03  0.00000000e+00 -1.59094715e-01  0.00000000e+00]
 [ 3.36655545e-03  0.00000000e+00 -8.58302996e-02  0.00000000e+00]
 [ 1.61442593e-03  0.00000000e+00 -4.11597740e-02  0.00000000e+00]
 [ 7.25808008e-04  0.00000000e+00 -1.85044683e-02  0.00000000e+00]
 [ 3.13253995e-04  0.00000000e+00 -7.98640765e-03  0.00000000e+00]
 [ 1.31442867e-04  0.00000000e+00 -3.35113467e-03  0.00000000e+00]
 [ 5.40284599e-05  0.00000000e+00 -1.37745508e-03  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [-6.05765966e-03  7.09551560e-04  1.54439976e-01 -7.93054556e-02]
 [-1.65697463e-03  1.47349972e-03  4.22445526e-02 -1.64690733e-01]
 [ 1.22347776e-03  1.74323292e-03 -3.11925539e-02 -1.94838387e-01]
 [ 2.08044434e-03  1.64840390e-03 -5.30409089e-02 -1.84239497e-01]
 [ 1.93406969e-03  1.38428736e-03 -4.93090886e-02 -1.54719609e

In [15]:
jaco_store = pd.DataFrame({'A1': gradient_c.T[0],
                          'A2': gradient_c.T[1],
                          'E1': gradient_c.T[2],
                          'E2': gradient_c.T[3]})
print(jaco_store)

          A1        A2        E1        E2
0   0.000000  0.000000  0.000000  0.000000
1   0.008675  0.000000 -0.221173  0.000000
2   0.006240  0.000000 -0.159095  0.000000
3   0.003367  0.000000 -0.085830  0.000000
4   0.001614  0.000000 -0.041160  0.000000
5   0.000726  0.000000 -0.018504  0.000000
6   0.000313  0.000000 -0.007986  0.000000
7   0.000131  0.000000 -0.003351  0.000000
8   0.000054  0.000000 -0.001377  0.000000
9   0.000000  0.000000  0.000000  0.000000
10 -0.006058  0.000710  0.154440 -0.079305
11 -0.001657  0.001473  0.042245 -0.164691
12  0.001223  0.001743 -0.031193 -0.194838
13  0.002080  0.001648 -0.053041 -0.184239
14  0.001934  0.001384 -0.049309 -0.154720
15  0.001482  0.001081 -0.037787 -0.120863
16  0.001034  0.000805 -0.026349 -0.089987
17  0.000683  0.000579 -0.017417 -0.064770
18  0.000000  0.000000  0.000000  0.000000
19 -0.002617 -0.000710  0.066733  0.079305
20 -0.004583 -0.001473  0.116850  0.164691
21 -0.004590 -0.001743  0.117023  0.194838
22 -0.00369

In [ ]:
k1_ext = value(model_un.k1)
k2_ext = value(model_un.k2)
timepoint = [0.0, 0.125, 0.25, 0.375, 0.5,0.625, 0.75, 0.875, 1.0]
CA = []
CB = []
CC = []
for i in timepoint:
    CA.append(value(model_un.C['CA',i]))
    CB.append(value(model_un.C['CB',i]))
    CC.append(value(model_un.C['CC',i]))
    
dCAdt = []
dCBdt = []
dCCdt = []

for j in range(9):   
    dCAdt.append(-k1_ext*CA[j])
    dCBdt.append(k1_ext*CA[j]-k2_ext*CB[j])
    dCCdt.append(k2_ext*CB[j])
print('dCAdt:', dCAdt)
print('dCBdt:', dCBdt)
print('dCCdt:', dCCdt)